In [ ]:
!open .

In [2]:
import pandas as pd
import requests
import csv

In [4]:
firm_code = 'A039490'
fs_url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701&gicode=' + firm_code
fs_page = requests.get(fs_url)
fs_tables = pd.read_html(fs_page.text)


In [7]:
fs_tables[0]

info1 = fs_tables[0]

info1.set_index(info1.columns[0])
info1.loc[6][1]

'25,662,745/ 0'

In [10]:

fs_tables[4]




,주주구분,대표주주수,보통주,지분율,최종변동일
0,최대주주등 (본인+특별관계자),1,10808047,42.12,2020/12/09
1,10%이상주주 (본인+특별관계자),1,3041927,11.85,2020/12/28
2,5%이상주주 (본인+특별관계자),1,2140842,8.34,2021/01/25
3,"임원 (5%미만 중, 임원인자)",6,4867,0.02,2020/10/12
4,자기주식 (자사주+자사주신탁),1,500000,1.95,2019/08/12
5,우리사주조합,1,4368,0.02,2018/09/28


In [9]:

fs_tables[11]

IFRS(연결)    Annual                                             \
      IFRS(연결)   2015/12   2016/12    2017/12    2018/12    2019/12   
0        순영업수익   4663.00   4735.00    5892.00    6181.00    8651.00   
1         영업이익   2414.00   2307.00    3158.00    2890.00    4737.00   
2   영업이익(발표기준)   2414.00   2307.00    3158.00    2890.00    4737.00   
3        당기순이익   1900.00   1802.00    2416.00    1932.00    3620.00   
4      지배주주순이익   1897.00   1799.00    2402.00    1932.00    3628.00   
5     비지배주주순이익      3.00      3.00      14.00       0.00      -7.00   
6         자산총계  61522.00  88571.00  118566.00  182397.00  233736.00   
7         부채총계  50424.00  76090.00  103094.00  161739.00  211152.00   
8         자본총계  11098.00  12480.00   15472.00   20658.00   22584.00   
9       지배주주지분  11031.00  12437.00   15246.00   20415.00   22340.00   
10     비지배주주지분     68.00     43.00     226.00     243.00     245.00   
11         자본금   1105.00   1105.00    1105.00    1270.00    1270.00   
12        부채비율    454.33    609.68     666.33     782.95     934.95   
13         유보율    898.25   1025.57    1279.76    1507.95    1690.39   
14       영업이익률     29.32     24.45      25.97      13.46      15.50   
15    지배주주순이익률     23.04     19.07      19.75       9.00      11.87   
16         ROA      3.50      2.40       2.33       1.28       1.74   
17         ROE     18.78     15.33      17.36      10.84      16.97   
18      EPS(원)   8585.00   8142.00   10870.00    7753.00   14287.00   
19      BPS(원)  49913.00  56278.00   68988.00   80397.00   89519.00   
20      DPS(원)    700.00    850.00    1300.00    1500.00    2000.00   
21         PER      7.22      8.84       8.06      10.09       5.56   
22         PBR      1.24      1.28       1.27       0.97       0.89   
23       발행주식수  22100.00  22100.00   22100.00   22100.00   22100.00   
24       배당수익률      1.13      1.18       1.48       1.92       2.52   

                                     
   2020/12(E) 2021/12(E) 2022/12(E)  
0    13997.00   12888.00   12381.00  
1     9181.00    7946.00    7391.00  
2         NaN        NaN        NaN  
3     6687.00    5848.00    5519.00  
4     6778.00    6274.00    5785.00  
5         NaN        NaN        NaN  
6   326815.00  354385.00  384698.00  
7   298862.00  321516.00  347217.00  
8    28021.00   33035.00   37428.00  
9    27887.00   32877.00   37563.00  
10     134.00     158.00    -135.00  
11    1270.00    1270.00    1270.00  
12    1066.55     973.25     927.70  
13        NaN        NaN        NaN  
14      65.60      61.66      59.69  
15      48.42      48.68      46.72  
16       2.39       1.72       1.49  
17      26.99      20.65      16.43  
18   26410.00   24446.00   22542.00  
19  110196.00  129641.00  147897.00  
20    2796.00    2979.00    3036.00  
21       5.96       6.44       6.99  
22       1.43       1.21       1.06  
23        NaN        NaN        NaN  
24        NaN        NaN        NaN